In [1]:
#verificar com o ayres [[]] aninhado

import json
import re

data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Numero de documentos: {len(data)}')

Numero de documentos: 11225


In [2]:
#print(data[0].keys())

In [3]:
#print(data[0]['title'])

In [4]:
#print(data[0]['body'])

---

Como vocês podem ver, o texto está cheio de caracteres de estruturação de documento.

---

**Atividade:** Consulte a documentação da Wikipedia para descobrir o que são as marcações `[[`, `]]`, `{{` e ```}}```

**R:** O que está entre [[ ]] é um hypertexto. o que está entre {{}} é um supertexto (ex: aaaa², o 2 é um supertexto)

---

Vamos explorar o corpus para minerar itens de interesse.

**Exercicio:** Faça uma função que retorna uma lista com todos os links html de um texto deste corpus.

In [5]:
# def find_links(text):
#     return re.findall(r'\[\[.+?\]\]', text)
# find_links(" evoluir.thumb|esquerda|300px|Primeira edi\u00e7\u00e3o do Unua Libro, em [[L\u00edngua russa|russo]]O primeiro livro sobre o esperanto foi")

In [6]:
#print(find_links(data[2]['body']))

**Exercicio:** Faça uma função que recebe um texto do corpus e substitui todas as ocorrências de links da Wikipedia (textos entre os tags `[[` e `]]`) por texto simples, e retorna o documento limpo. Por exemplo, quando encontrar algo como `[[Etimologia|Etimologicamente]]` substituir por `Etimologicamente`. Você pode supor que os links não são *aninháveis*, como por exemplo `[[blablabla[[etc]]blebleble]]`.

In [7]:
def replace_links(texto):
    pattern = r'''
        \[\[
        (?:
            [^|]*?\|
        )*?
        (
            [^|]*?
        )
        \]\]
    '''
    replacement = r'\1' # primeiro grupo de captura
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(replacement, texto)

**Exercicio**: Faça uma função que recebe um texto do corpus e remove todas as ocorrências de referências (textos entre os tags `<ref>` e `</ref>`).

In [8]:
def remove_ref(text):
    return re.sub(r'<ref(?:.*)?>.*?(?:</ref>)?', '', text)

In [9]:
def remove_math(text):
    return re.sub(r'<math(?:.*)?>.*?(?:</math>)?', '', text)

In [10]:
def remove_small(text):
    return re.sub(r'<.*?>(.*?)<.*?>', r'\1', text)

In [11]:
#replace vss sub
def limpa_aspas(texto):
    pattern = r"""(['"]+)(.*?)\1"""
    matcher = re.compile(pattern)
    return matcher.sub(r'\2', texto)

In [12]:
def remove_endline(text):
    return text.replace('\n','')

In [13]:
def remove_double_equal(text):
    return text.replace('==','')

In [14]:
#Remove {}
def limpa_chaves(texto):
    pattern = r"{.*?}"
    matcher = re.compile(pattern)
    return matcher.sub('', texto)


In [15]:
def limpa_colchete(texto):
    pattern = r"\[.*?\]"
    matcher = re.compile(pattern)
    return matcher.sub('', texto)

In [16]:
def limpa_obs(texto):
    pattern = r"<!--.*?-->"
    matcher = re.compile(pattern)
    return matcher.sub('', texto)

In [21]:
def remove_ast(texto):
    texto = texto.replace("*", "")
    return texto

In [28]:
def remove_px(text):     
    return re.sub(r"(?:\d+x)?\d+px", '', text)

remove_px("o problema.624pxDurante a adolescênci")

'o problema.Durante a adolescênci'

**Exercicio:** Faça uma função que recebe um texto do corpus e remove todos os templates (textos entre os tags `{{` e `}}`). Este exercício é desafiante: *os templates podem ser aninhados*! E agora, como proceder?

In [18]:
# def remove_templates(text):
#     a = text
#     while re.search(r'\{\{[^\{\{\}\}]*\}\}', a):
#         to_replace = re.findall(r'\{\{[^\{\}]*\}\}', a)
#         for i in to_replace:
#             a = a.replace(i, '')
    
#     return a

def remove_templates(texto):
    conta = 0
    spans_proibidos = []
    for item in re.finditer(r'{{|}}', texto):
        if item[0] == '{{':
            if conta == 0:
                inicio = item.span()[0]
            conta += 1
        else:
            conta -= 1
            if conta == 0:
                fim = item.span()[1]
                spans_proibidos.append((inicio, fim))

    texto_limpo = ''
    inicio = 0
    for span in spans_proibidos:
        fim, novo_inicio = span
        texto_limpo += texto[inicio:fim]
        inicio = novo_inicio
    
    texto_limpo += texto[inicio:]
    return texto_limpo



**Exercicio:** Usando os codigos desenvolvidos acima, faça uma função que recebe um texto do corpus e retorna a sua versão limpa.

In [19]:
def clear_text(text):
    clean_links = replace_links(text)
    remove_quotes = limpa_aspas(clean_links)
    remove_ref_ = remove_ref(remove_quotes)
    remove_endline_ = remove_endline(remove_ref_)
    remove_double_equal_ = remove_double_equal(remove_endline_)
    remove_templates_ = remove_templates(remove_double_equal_)
    remove_math_ = remove_math(remove_templates_)
    remove_small_ = remove_small(remove_math_)
    remove_colchete_ = limpa_colchete(remove_small_)
    limpa_obs_ = limpa_obs(remove_colchete_)
    remove_ast_ = remove_ast(limpa_obs_)
    remove_px_ = remove_px(remove_ast_)
    return limpa_chaves(remove_px_)

**Exercicio:** Limpe todos os documentos e explore os resultados para ver o que mais dá para limpar. Nosso objetivo é ter uma coletânea de textos limpos para poder criar um vocabulário da língua portuguesa!

In [20]:
import json
with open('dump_small_clean.jsonln', 'w', encoding="utf8") as file:
    for i in range(10):
        json.dump({"body": clear_text(data[i]['body']), "title": data[i]["title"]}, file, ensure_ascii=False)

# Criando um tokenizador

Agora que temos um conjunto de documentos limpos, vamos analisar esses documentos para:

- Separar os documentos em palavras

- Separar os documentos em frases

**Exercicio:** Faça uma função que separa os documentos em palavras usando simplesmente os espaços em branco como indicador de separação.

**Exercicio:** Agora analise os resultados obtidos: essa separação em palavras está perfeita? Certamente não:

- Um problema óbvio é o seguinte: palavras ao final de uma sentença podem carregar consigo a pontuação da frase. 

- Mais ainda, algumas palavras no meio das sentenças podem ter vírgula e ponto-e-vírgula juntos.

- Sequências de emojis podem aparecer, e cada emoji deve ser considerado como uma palavra separada

Faça uma função que recebe um documento e faz uma separação melhor.

In [77]:
scanner=re.Scanner([
  (r"[0-9]+",       lambda scanner,token:("INTEGER", token)),
  (r"[\w]+",      lambda scanner,token:("IDENTIFIER", token)),
  (r"[,.\!\?\=-]+",        lambda scanner,token:("PUNCTUATION", token)),
  (r"[()\[\]{}\\]+",        lambda scanner,token:("TRASH", token)),
  (r"\s+", None), # None == skip token.
], re.UNICODE)
 
results, remainder=scanner.scan("45 pigéons, 23 cows, 11 spiders.")
print(results)



[('INTEGER', '45'), ('IDENTIFIER', 'pigéons'), ('PUNCTUATION', ','), ('INTEGER', '23'), ('IDENTIFIER', 'cows'), ('PUNCTUATION', ','), ('INTEGER', '11'), ('IDENTIFIER', 'spiders'), ('PUNCTUATION', '.')]
